<a href="https://colab.research.google.com/github/JoshuaZheng0/Backtest-Trading-Sim/blob/main/RSI-trading-backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Model Training and preprocessing libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
#Performance evaluation libraries

dataset_train = pd.read_csv("4-23 to 7-16 5 min data NQ.csv")
len(dataset_train)

20472

In [ ]:
#drop the Adj close column since it gives the same values as close
# create a new column that represents the percentage in price change
dataset_train = dataset_train[['time','open', 'high', 'low','close']]
dataset_train['PC%'] = [0] * dataset_train.shape[0]
dataset_train['PC%'] = (dataset_train['close'].diff()/dataset_train['open'])*100
#create rsi column
def rsi(ohlc: dataset_train, period: int = 14, round_rsi: bool = True):
    "Implements the RSI indicator as defined by TradingView on March 15, 2021."
    delta = ohlc["close"].diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return np.round(rsi, 2) if round_rsi else rsi


dataset_train['rsi'] = [0] * dataset_train.shape[0]
dataset_train['rsi'] = rsi(dataset_train)
dataset_train["PC%"]=dataset_train["PC%"].fillna(0)
dataset_train["rsi"]=dataset_train["rsi"].fillna(0)
# #the feature class represents tesla's change the following day
# dataset_train['class'] = [0] * dataset_train.shape[0]
# dataset_train['class'] = dataset_train['PC%'].apply(lambda x : x >= 0)
#if tesla grew class will be indicated as 1
# #if tesla plumeted class will be indicated as 0
# dataset_train["class"] = dataset_train["class"].replace(True, 1)
# dataset_train["class"] = dataset_train["class"].replace(False, 0)
# dataset_train['class'] = dataset_train['class'].shift(-1)
# dataset_train["class"]=dataset_train["class"].fillna(0)
# #if tesla grows for the nesxt 3 days next
# dataset_train['nextThreeDays'] = [0] * dataset_train.shape[0]
# dataset_train['nextThreeDays'] = (dataset_train['class']+dataset_train['class'].shift(-1)+dataset_train['class'].shift(-2))//3
# dataset_train["nextThreeDays"]=dataset_train["nextThreeDays"].fillna(0)
# dataset_train["nextThreeDays"].sum()
# #the growth of tesla in the past three days
# dataset_train['pastThreeDays'] = [0] * dataset_train.shape[0]
# dataset_train['pastThreeDays'] = (dataset_train['class'].shift(1)+dataset_train['class'].shift(2)+dataset_train['class'].shift(3))//3
# dataset_train["pastThreeDays"]=dataset_train["pastThreeDays"].fillna(0)
# dataset_train["pastThreeDays"].sum()
dataset_train

,time,open,high,low,close,PC%,rsi
0,1680472800,13262.00,13275.50,13247.00,13251.00,0.000000,0.00
1,1680473100,13250.00,13258.25,13244.25,13246.25,-0.035849,0.00
2,1680473400,13245.75,13247.75,13236.25,13242.75,-0.026424,0.00
3,1680473700,13242.75,13247.75,13235.00,13242.50,-0.001888,0.00
4,1680474000,13243.00,13248.50,13243.00,13246.50,0.030205,36.19
...,...,...,...,...,...,...,...
20467,1689559500,15683.25,15685.75,15681.00,15685.25,0.017535,59.21
20468,1689559800,15685.00,15686.50,15682.00,15682.75,-0.015939,54.87
20469,1689560100,15682.75,15684.25,15681.25,15682.50,-0.001594,54.44
20470,1689560400,15682.00,15683.00,15681.25,15682.25,-0.001594,53.98


In [ ]:
#the rsi, open, and change colums are shifted down by 15 days
#since rsi requires the previous 15 days to calculate
#dont worry that the last 15 values in the rsi array are null
#they occur because of the initial shift.
rsi = dataset_train['rsi'].shift(-15)
open = dataset_train['open'].shift(-15)
change =  dataset_train['PC%'].shift(-15)
time = dataset_train['time'].shift(-15)

sum_of_all_percentages = 0



In [ ]:
len(rsi)

20472

In [ ]:
from itertools import islice
import collections

def consume(iterator, n):
    "Advance the iterator n-steps ahead. If n is none, consume entirely."
    # Use functions that consume iterators at C speed.
    if n is None:
        # feed the entire iterator into a zero-length deque
        collections.deque(iterator, maxlen=0)
    else:
        # advance to the empty slice starting at position n
        next(islice(iterator, n, n), None)

In [ ]:
                                                      #variables and explinations
#create an empty array to store values in
array = []
array.clear()

#the max percent of correct predictions
max_percent = 0

#the rsi value this max percent occured at
best_rsi = 0
best_array = []

#under the simulation how much money would be made
money_made_per_cycle = 0
total_money_made = 0

#the timestamp of the occuring pattern for checking if the program functions
timestamp = []
timestamp.clear()

#the number of candles to hold after buying
hold_length_tuner = 2
                                                                                       #optimization for rsi
#the rsi runer runs through 30-70 to determine the best rsi threshold
for rsi_tuner in range (99,100):

  #runs a loop through each value in the dataset other than the first 3 days
  numbers = iter(range(2,len(rsi)))
  for i in numbers:
    #if rsi is less than threshhold, the price change today is positive,
    #the price change yesterday was negative
    #and the price change of the day before yesterday was negative
    if rsi[i] <= rsi_tuner and change[i] < 0 and change[i-1] < 0 and change[i-2] < 0 and (i+hold_length_tuner+1)<len(rsi)-15:

      cash_flow_change = open[i+1+hold_length_tuner]-open[i+1] #the number represents the number of days/30 mins/5 min cycles held

      if cash_flow_change > 0:
        array.append(True) #True if the function made money

      else:
        array.append(False)#False if the function lost money

      timestamp.append(time[i])
      money_made_per_cycle += cash_flow_change
      #consume(numbers, hold_length_tuner+1) #skips next three days of the inner for loop




  print(rsi_tuner)
  print((array.count(True)/(array.count(True)+array.count(False)))*100)
  sum_of_all_percentages += (array.count(True)/(array.count(True)+array.count(False)))*100
  print(array)
  print(timestamp)
  print(money_made_per_cycle)
  print("\n")
  print(array.count(True)/len(array))
  print(len(array))





  #this code determines the max/best values based on percent of times the function made moeny
  # if (array.count(True)+array.count(False)) != 0: #makes sure denominator isnt 0
  #   if (array.count(True)/(array.count(True)+array.count(False))) > max_percent and len(array) >10:
  #     max_percent = (array.count(True)/(array.count(True)+array.count(False))) #finds max percent
  #     best_rsi = rsi_tuner                                                     #stores best rsi threshold in a variable
  #     best_array = array.copy()                                                #stores the array for the best rsi threshold
  #     best_timestamp = timestamp.copy()                                        #stores the current timestamp when money is maximized
  #     total_money_made = money_made_per_cycle                                  #stores the total money made in one cycle

#  this code determines the max/best values based on the total amount of money the function made.
  # if money_made_per_cycle > total_money_made:
  #     max_percent = (array.count(True)/(array.count(True)+array.count(False))) #finds max percent
  #     best_rsi = rsi_tuner                                                     #stores best rsi threshold in a variable
  #     best_array = array.copy()                                                #stores the array for the best rsi threshold
  #     best_timestamp = timestamp.copy()                                        #stores the current timestamp when money is maximized
  #     total_money_made = money_made_per_cycle                                  #stores the total money made in one cycle

 # clears variables after 1 cycle of rsi change
  array.clear()
  timestamp.clear()
  money_made_per_cycle = 0

# best_timestamp = [int(x) for x in best_timestamp] #changes float into int

# print(best_rsi)
# print(max_percent*100)
# print(best_array)
# print(best_timestamp)
# print(total_money_made)


99
52.07014125669751
[True, False, False, False, False, False, True, True, True, True, False, False, True, False, False, False, True, True, True, False, False, True, True, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, False, False, False, False, False, False, False, True, True, False, False, True, False, False, False, True, True, True, True, True, False, False, False, True, False, True, True, False, False, False, False, True, False, False, False, True, True, False, False, True, True, True, True, True, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, False, True, False, False, True, True, False, False, True, True, False, False, False, True, False, True, False, True, True, False, True, True, True, True, False, True, True, False, False, False, False, True, False, False, True, True, False, True, True, True, True, True, True, False, False, True, True, False, True, False, False, False, True, 

In [ ]:
#probability trading
arr = [True, True, False, False, False] #input value, left is most recent
current_rsi = 58
 #input value

#create an empty array to store valu|es in
array = []
array.clear()

#the number of candles to hold after buying

for hold_length_tuner in range(1,6):
  print("\n\n")
  print("HOLD FOR " + str(hold_length_tuner) + " INTERVALS")                                                                                    #optimization for rsi
  #the rsi runer runs through 30-70 to determine the best rsi threshold
  array.clear()
  for rsi_tuner in range (current_rsi,current_rsi+1):

    #runs a loop through each value in the dataset other than the first 3 days
    numbers = iter(range(4,len(rsi)))
    for i in numbers:
      #if rsi is less than threshhold, the price change today is positive,
      #the price change yesterday was negative
      #and the price change of the day before yesterday was negative
      if arr[0] is True:
        inequality_1 = change[i] > 0
      else:
        inequality_1 = change[i] < 0
      if arr[1] is True:
        inequality_2 = change[i-1] > 0
      else:
        inequality_2 = change[i-1] < 0
      if arr[2] is True:
        inequality_3 = change[i-2] > 0
      else:
        inequality_3 = change[i-2] < 0
      if arr[3] is True:
        inequality_4 = change[i-3] > 0
      else:
        inequality_4 = change[i-3] < 0
      if arr[4] is True:
        inequality_5 = change[i-4] > 0
      else:
        inequality_5 = change[i-4] < 0
      if rsi[i] <= rsi_tuner+5 and inequality_1 and inequality_2 and inequality_3 and inequality_4 and inequality_5 and (i+hold_length_tuner+1)<len(rsi)-15 and rsi[i] >= rsi_tuner-5:

        cash_flow_change = open[i+1+hold_length_tuner]-open[i+1] #the number represents the number of days/30 mins/5 min cycles held

        if cash_flow_change > 0:
          array.append(True) #True if the function made money

        else:
          array.append(False)#False if the function lost money
        consume(numbers, hold_length_tuner+1) #skips next three days of the inner for loop
    print(str(rsi_tuner) + " rsi")
    print("PAST 5 candles")
    print((array.count(True)/(array.count(True)+array.count(False)))*100)
    print("occured " + str(len(array)) + " times out of 20500")
    array = []
    array.clear()





  for rsi_tuner in range (current_rsi,current_rsi+1):

    #runs a loop through each value in the dataset other than the first 3 days
    numbers = iter(range(3,len(rsi)))
    for i in numbers:
      #if rsi is less than threshhold, the price change today is positive,
      #the price change yesterday was negative
      #and the price change of the day before yesterday was negative
      if arr[0] is True:
        inequality_1 = change[i] > 0
      else:
        inequality_1 = change[i] < 0
      if arr[1] is True:
        inequality_2 = change[i-1] > 0
      else:
        inequality_2 = change[i-1] < 0
      if arr[2] is True:
        inequality_3 = change[i-2] > 0
      else:
        inequality_3 = change[i-2] < 0
      if arr[3] is True:
        inequality_4 = change[i-3] > 0
      else:
        inequality_4 = change[i-3] < 0
      if rsi[i] <= rsi_tuner+5 and inequality_1 and inequality_2 and inequality_3 and inequality_4 and (i+hold_length_tuner+1)<len(rsi)-15 and rsi[i] >= rsi_tuner-5:

        cash_flow_change = open[i+1+hold_length_tuner]-open[i+1] #the number represents the number of days/30 mins/5 min cycles held

        if cash_flow_change > 0:
          array.append(True) #True if the function made money

        else:
          array.append(False)#False if the function lost money
        consume(numbers, hold_length_tuner+1) #skips next three days of the inner for loop


    print("PAST 4 candles")
    print((array.count(True)/(array.count(True)+array.count(False)))*100)
    print("occured " + str(len(array)) + " times out of 20500")

    array = []
    array.clear()


    #the rsi runer runs through 30-70 to determine the best rsi threshold
  for rsi_tuner in range (current_rsi,current_rsi+1):

    #runs a loop through each value in the dataset other than the first 3 days
    numbers = iter(range(3,len(rsi)))
    for i in numbers:
      #if rsi is less than threshhold, the price change today is positive,
      #the price change yesterday was negative
      #and the price change of the day before yesterday was negative
      if arr[0] is True:
        inequality_1 = change[i] > 0
      else:
        inequality_1 = change[i] < 0
      if arr[1] is True:
        inequality_2 = change[i-1] > 0
      else:
        inequality_2 = change[i-1] < 0
      if arr[2] is True:
        inequality_3 = change[i-2] > 0
      else:
        inequality_3 = change[i-2] < 0

      if rsi[i] <= rsi_tuner+5 and inequality_1 and inequality_2 and inequality_3 and (i+hold_length_tuner+1)<len(rsi)-15 and rsi[i] >= rsi_tuner-5:

        cash_flow_change = open[i+1+hold_length_tuner]-open[i+1] #the number represents the number of days/30 mins/5 min cycles held

        if cash_flow_change > 0:
          array.append(True) #True if the function made money

        else:
          array.append(False)#False if the function lost money
        consume(numbers, hold_length_tuner+1) #skips next three days of the inner for loop

  print("PAST 3 candles")
  print((array.count(True)/(array.count(True)+array.count(False)))*100)
  print("occured " + str(len(array)) + " times out of 20500")
  array = []
  array.clear()
    #the rsi runer runs through 30-70 to determine the best rsi threshold
  for rsi_tuner in range (current_rsi,current_rsi+1):

    #runs a loop through each value in the dataset other than the first 3 days
    numbers = iter(range(3,len(rsi)))
    for i in numbers:
      #if rsi is less than threshhold, the price change today is positive,
      #the price change yesterday was negative
      #and the price change of the day before yesterday was negative
      if arr[0] is True:
        inequality_1 = change[i] > 0
      else:
        inequality_1 = change[i] < 0
      if arr[1] is True:
        inequality_2 = change[i-1] > 0
      else:
        inequality_2 = change[i-1] < 0

      if rsi[i] <= rsi_tuner+5 and inequality_1 and inequality_2 and (i+hold_length_tuner+1)<len(rsi)-15 and rsi[i] >= rsi_tuner-5:

        cash_flow_change = open[i+1+hold_length_tuner]-open[i+1] #the number represents the number of days/30 mins/5 min cycles held

        if cash_flow_change > 0:
          array.append(True) #True if the function made money

        else:
          array.append(False)#False if the function lost money
        consume(numbers, hold_length_tuner+1) #skips next three days of the inner for loop++

  print("PAST 2 candles")
  print((array.count(True)/(array.count(True)+array.count(False)))*100)
  print("occured " + str(len(array)) + " times out of 20500")





HOLD FOR 1 INTERVALS
58 rsi
PAST 5 candles
48.1203007518797
occured 133 times out of 20500
PAST 4 candles
50.74183976261127
occured 337 times out of 20500
PAST 3 candles
48.65900383141762
occured 783 times out of 20500
PAST 2 candles
48.090909090909086
occured 1100 times out of 20500



HOLD FOR 2 INTERVALS
58 rsi
PAST 5 candles
45.86466165413533
occured 133 times out of 20500
PAST 4 candles
48.6646884272997
occured 337 times out of 20500
PAST 3 candles
48.28060522696011
occured 727 times out of 20500
PAST 2 candles
48.96755162241888
occured 1017 times out of 20500



HOLD FOR 3 INTERVALS
58 rsi
PAST 5 candles
50.37593984962406
occured 133 times out of 20500
PAST 4 candles
50.153846153846146
occured 325 times out of 20500
PAST 3 candles
52.41581259150805
occured 683 times out of 20500
PAST 2 candles
52.73311897106109
occured 933 times out of 20500



HOLD FOR 4 INTERVALS
58 rsi
PAST 5 candles
43.18181818181818
occured 132 times out of 20500
PAST 4 candles
44.479495268138805
occured 

In [ ]:
#For tesla stock analysed based on its daily values. the daily strategy of buying right after the pattern down down up at any rsi value will guarentee you to make money
#however these instances are very rare as they only occur 28 out of 300 days of trading. so snatch at the pattern when it appears.
#this estimates to about 300 dollars a year, which is very minimal

In [ ]:
#for NQ on a 5 min interval, it seems the best time to trade is when the rsi is between 35-45 or anything lower thsn that. The best interval to hold the stock is for 15 to 25 minutes
#normally its best to hold the stock for only 15 min. it completely depends on circumstance. the overall percentage of the stock making money is 53-57 percent which is relatively low.
#but if the right strategy is estabilshed and the trading is systematic the strategy will make money.